# Seq2Seq z atencją - analiza modelu

Cześć! 
Ten notebook stworzony jest do eksploracji i praktycznego wykorzystania możliwości modelu *seq2seq z mechanizmem atencji*!. Dzięki przerobieniu wszystkich proponowanych przez nas zadań, dowiesz się jak: 
- wczytać przetrenowany model 
- wygenerować predykcje z modelu seq2seq
- wyciągnąć z modelu reprezentację dokumentów i porównać je ze sobą
- wyciągnąć z modelu wagi atencji i odnaleźć dla każdego ogłoszenia najważniejsze słowa
- sprawdzić które słowa są najbardziej znaczące dla każdej z naszych kategorii

Pracujemy na zbiorze z treściami ofert pracy. Oprócz treści oferty pracy, udostępniona jest również nazwa danego stanowiska (nie będziemy jej jednak wykorzystywać podczas tych ćwiczeń). Model został wytrenowany na zbiorze treningowym (17000 wpisów) i zwalidowany (3000 wpisów). Zbiór testowy, na którym będziemy pracować zawiera 1000 wpisów. <br>
W każdym ze wspomnianych zbiorów znajduje się 35 kategorii, które mogą przyjąć oferty. Niektóre z nich są do siebie dość podobne (np. "IT - Rozwój oprogramowania" i "IT - Administracja", a niektóre dość różne np. "Marketing" i "Inżynieria"). Skrypty użyte do przetwarzania danych i wytrenowania ich dostępne są w pliku `Modul2 [seq2seqAttn] Przetwarzanie danych i trenowanie modelu`.
<br>

Najpierw zaimportujmy potrzebne biblioteki i zdefiniujmy istotne zmienne i funkcje.

In [3]:
import pandas as pd
import pickle
import numpy as np
from keras.models import load_model, Model
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional, Input, Concatenate
from keras.initializers import Constant
from layers.attention import AttentionDecoder 
from utils.text_helpers import *
from utils.analysis_helpers import *
import matplotlib.pyplot as plt

In [4]:
#wczytanie danych

#data_focus - na której kolumnie się skupiamy; job_content - treść oferty; job_name - nazwa stanowiska
#max_number_words - jaka jest wielkość naszego słownika?
#max_seq_len - ile maksymalnie tokenów z każdej oferty pracy rozważamy? 

dataset_details = {
    'data_focus': 'job_content',
    'max_number_words': 20000,
    'max_seq_len': 250
}
data = Data(dataset_details['data_focus'])
data.preprocess_data(max_number_words=dataset_details['max_number_words'], max_seq_len=dataset_details['max_seq_len'])


#definicja argumentów sieci
#embedding_dim - wymiarowość każdego embeddingu
#embeddings - klucz zawierający macierz embeddingów dla naszego słownika
#train_embeddings - czy trenować embeddingi
#latent_dim - wymiar ukrytych stanów w LSTM/AttentionDecoder
#return_probabilities - klucz określający czy jesteśmy zainteresowani predykcjami (False) czy wagami atencji (True)
#num_decoder_tokens - liczba kategorii + 2 (dekodujemy kategorię + znak, że predykcja jest zakończona)
args = {
    'embedding_dim': 300,
    'embeddings': generate_embedding_matrix(load_word_embeddings('./data/wiki.pl.vec'), data.tokenizer.word_index),
    'train_embeddings': True,
    'latent_dim': dataset_details['max_seq_len'],
    'dropout': 0.5, 
    'num_decoder_tokens': 37,
    'batch_size': 500, 
    'num_epochs': 5,
    'return_probabilities': False,
}

network = Network(args)

In [7]:
#dzięki tej komendzie łatwo zobaczymy jakie własności zawiera nasz obiekt
print(data.__dict__.keys())

dict_keys(['data_focus', 'training_set', 'test_set', 'raw_train_X', 'raw_val_X', 'raw_train_y', 'raw_val_y', 'raw_test_X', 'raw_test_y', 'tokenizer', 'cleaner', 'unique_class_names', 'train_X', 'val_X', 'test_X', 'train_y', 'train_y_input', 'train_y_output', 'val_y', 'val_y_input', 'val_y_output', 'test_y', 'test_y_input', 'test_y_output'])


***Zadanie 1***. Na podstawie części teoretycznej i notebooka `Modul2 [seq2seqAttn] Przetwarzanie danych i trenowanie modelu`, wypełnij strukturę sieci, która odpowiada za encoder. W ramach wykonania zadania, do sieci powinny być dodane dwa bloki - jeden odpowiadający za embeddingi inputu, a drugi za dwukierunkowy LSTM jako encoder.

In [ ]:
def seq2seq(network):
    encoder_inputs = Input(shape=(None,), name='encoder_input')
    #dodaj warstwę embeddingów i encodera, a następnie wczytaj model z pliku
    embedding_layer = # name="embeddings"
    embedded_sequences = #
    encoder = # name="encoder"
    states =  #
    outputs_true = Input(shape=(None, None,), dtype='int64', name='decoder_input')
    decoder_outputs = AttentionDecoder(network.latent_dim*2, network.num_decoder_tokens, \
        return_probabilities=network.return_probabilities, name='attention')([states, outputs_true], \
        use_teacher_forcing=False)
    model = Model([encoder_inputs, outputs_true], decoder_outputs)
    return model

def load(path, network, weights=True):
    if weights: 
        model = seq2seq(network)
        print('model created')
        model.load_weights(path, by_name=True)
    else:
        model = load_model(path, custom_objects={"AttentionDecoder": AttentionDecoder})
    return model

Po pomyślnym wykonaniu zadania 1, wczytanie modelu powinno być bezproblemowe.

In [ ]:
model = load('./data/model_seq2seq_job_content.h5',network, weights=False)
model.summary()

Jako że mamy już wczytane dane oraz model, możemy stworzyć kolejny moduł do generowania predykcji. Predykcje w seq2seq różnią się nieco od tych w standardowych problemach klasyfikacyjnych. Nie przewidujemy pojedynczej klasy, tylko sekwencję (dla `n` kroków w czasie). <br><br>
***Zadanie 2*** Uzupełnij moduł Predictions o funkcję `get_predictions`. Jej celem jest wygenerowanie predykcji na podanych inputach, a następnie przetworzenie jej w ten sposób, by dla każdego kroku w czasie otrzymać przewidzianą dla niego klasę.  

In [6]:
class Predictions:
    def __init__(self, model, inputs, class_names):
        self.model = model
        self.inputs = inputs
        self.class_names = class_names
        self.raw_predictions = self.get_predictions(self.model, self.inputs, self.class_names)
        self.predicted_labels = self.transform_predictions(self.raw_predictions, self.class_names)
        self.true_labels = data.raw_test_y
        
    def transform_predictions(self, predictions, class_names):
        #funkcja usuwająca predykcje z tokenami odnoszącymi się do końca sekwencji 
        x = [[class_names[item] for item in sublist] for sublist in predictions]
        x = [a[:a.index('\n')] if '\n' in a else a for a in x ]
        return x

    def get_predictions(self, model, inputs, class_names):
        #uzupełnij funkcję
        
        return predictions

In [ ]:
predictions = Predictions(model, [data.test_X, data.test_y_input], data.unique_class_names)

Sprawdźmy skuteczność naszego modelu na zbiorze testowym (0.93 na zbiorze walidacyjnym).

In [ ]:
correct_values = [i for i,x in enumerate(predictions.predicted_labels) if x[0]==data.raw_test_y.values[i]]
print(len(correct_values)/len(predictions.predicted_labels))

Model radzi sobie całkiem nieźle. Ale jeszcze wiele możemy zrobić, żeby poznać mechanizmy jego działania. Sprawdźmy które klasy najczęściej są ze sobą mylone - `confusion matrix`.

**Zadanie 3** Zaimplementuj confusion matrix. 
- Zredukuj liczbę klas, żeby nie brać pod uwagę tokenów otwierających i kończących predykcję
- użyj funkcji `create_matrix_from_labels` z pliku `utils.analysis_helpers` do stworzenia matrixowej reprezentacji prawdziwych i przewidzianych kategorii(osobno)
- użyj funkcji `cooccurrence_matrix` z pliku `utils.analysis_helpers` do stworzenia confusion matrix (jaki jest input do tej funkcji?) 
- użyj funkcji `plot_matrix` z pliku `utils.analysis_helpers` do pokazania confusion matrix

In [ ]:
reduced_class_names = #
true_matrix = #
pred_matrix = #
cooc_matrix = #

Nasz model jest przydatny nie tylko do skutecznego przewidywania kategorii, ale może być również wykorzystany w wyszukiwaniu podobnych ofert pracy. <br> 
W ramach tego ćwiczenia utniemy sieć przed dekoderem i dla każdego wpisu otrzymamy jego reprezentację w przestrzeni wektorowej (po przejściu przez warstwę Embedding i Bidirectional(LSTM)). Następnie wybierzemy kilka ofert, dla których znajdziemy inne, najbardziej do nich podobne oferty. 

***Zadanie 4*** 
Wydziel z modelu reprezentację wektorową dokumentów. Aby to zrobić: 
- zidentyfikuj co jest inputem i outputem naszej "zredukowanej sieci" (zapisz nazwy warstw)
- stwórz kolejny, zredukowany model
- wygeneruj na nim predykcje
- sprawdź kształt wyniku. Coś nie pasuje, prawda?
- pozbądź się środkowego wymiaru poprzez uśrednienie. Każdy dokument powinien być reprezentowany poprzez jedną liczbę dla jednego timestepu (zauważ, że prawdopodobnie ostatni wymiar to nasze latent_dim*2 (mamy do czynienia z Bidirectional LSTM)
- ponownie sprawdź kształt wyniku
- **4a** Wygeneruj i przedstaw za pomocą wykresu reprezentację 2D naszych dokumentów.
- **4b** Porównaj ze sobą graficznie miejsca reprezentacji dwóch podobnych kategorii (np. IT - Rozwój oprogramowania i IT Administracja, albo Sprzedaż i Obsługa klienta)
- **4c** Porównaj ze sobą graficznie miejsca reprezentacji dwóch różnych kategorii (np. Marketing i Inżynieria)


In [ ]:
model_names = [x.name for x in model.layers]
input_reduced = #jaki jest nasz input
output_reduced = #jaki jest nasz output
model_reduced = Model(inputs=input_reduced, outputs=output_reduced)
vector_representations = model_reduced.predict(data.test_X)

#uśrednienie 
vector_representations_avgd = #

In [ ]:
#4a
from sklearn.manifold import TSNE

tsne= TSNE(n_components=2, random_state=42, verbose=2, perplexity=120)
reduced_vector_representations = tsne.fit_transform(vector_representations)

colors_categories = {x: tuple(np.random.rand(3,1).reshape(-1)) for x in data.unique_class_names}
main_category = [tuple(colors_categories[x]) for x in data.raw_test_y]

plt.figure(figsize=(15,10))
plt.scatter(reduced_vector_representations[:,0], reduced_vector_representations[:,1], color=main_category)

In [ ]:
#4a - zrób to samo, tylko że dla przewidzianej kategorii (nie prawdziwej)

In [ ]:
#4b i 4c
cat_a = # kategoria 1
cat_b = #kategoria 2
similar_categories_ind = # uzupełnij
plt.scatter(reduced_vector_representations[similar_categories_ind,0], reduced_vector_representations[similar_categories_ind, 1], color=np.array(main_category)[similar_categories_ind], alpha=0.6)


**4.4** Oprócz reprezentacji wizualnej, wyszukamy najbardziej podobne treści ofert pracy za pomocą KNN (k-nearest neighbors). Napisz funkcję, która będzie wyświetlać n najbliższych sąsiadów danego wpisu. 

In [ ]:
from sklearn.neighbors import NearestNeighbors

#fitujemy dla najbliższych 5 sąsiadów
nnb = NearestNeighbors(n_neighbors=5)
nnb.fit(vector_representations)

def get_nearest_offers(idx, data):
    vec = vector_representations[idx]
    distances, indices = nnb.kneighbors(vec)
    similar = []
    #uzupełnij i posortuj po odległości
    return similar

## W końcu atencja...
Kolejne dwa zadania będą wykorzystywać wyniki uzyskane dzięki mechanizmowi atencji. Jako że w tym zadaniu operujemy na przewidywanej sekwencji składającej się z jednego elementu, a także treści ofert pracy charakteryzują się znaczną długością, wizualizacja atencji tak jak jest dostępna w typowych publikacjach z tego zakresu (NMT) jest ciężka do osiągnięcia. Mimo tego, możemy sprawdzić, które słowa zostały zidentyfikowane jako najważniejsze dla każdej oferty pracy. 
<br>
<br>
Najpierw wczytamy predykcje z wagami atencji. 

In [ ]:
network_probabilities = Network(args)
network_probabilities.return_probabilities = True
model_proba = load('./data/model_seq2seq_job_content.h5', network_probabilities)
probabilities = model_proba.predict([data.test_X, data.test_y_input])

Poeksploruj kształt outputu z probabilities. Co Ci mówi? <br>
***Zadanie 5*** Napisz funkcję, która dla dowolnej predykcji  będzie zwracała top_n najważniejszych słów wraz z wartością ich wag. 
- input: indeks oferty pracy
- najpierw znajdź dla niej predykcję
- posortuj po najważniejszych argumentach
- dla każdego z najważniejszych argumentów odnajdź jakie to słowo i jaka jest jego waga


In [ ]:
def display_most_significant_words(index, data, probabilities, predictions, top_n=10):
    #index - indeks wpisu
    #data - zmienna obiektu Data
    #probabilities - wygenerowane wagi atencji
    #predictions - moduł predykcji
    return 


In [ ]:
display_most_significant_words(300, data, probabilities, predictions.predicted_labels)

In [ ]:
from keras.preprocessing.text import text_to_word_sequence

#dodatkowy kod do graficznej wizualizacji atencji

def sequence2txt(data, ind, tokenizer):
    return text_to_word_sequence(tokenizer.sequences_to_texts([data[ind]])[0])

def visualize_attention(index, network, data, probabilities, predictions, predicted=True):
    print("Real labels: {}".format(data.test_y.values[index][1:-1]))
    plt.clf()
    f = plt.figure(figsize=(8, 8.5))
    ax = f.add_subplot(1, 1, 1)
    # add image
    if predicted:
        output = predictions
    else:
        output = data.test_y.values
    i = ax.imshow(np.squeeze(probabilities[index][:len(output[index]),:len(sequence2txt(data.test_X, index, data.tokenizer))], axis=-1), interpolation='nearest')

    # add colorbar
    cbaxes = f.add_axes([0.2, 0.1, 0.6, 0.04])
    cbar = f.colorbar(i, cax=cbaxes, orientation='horizontal')
    cbar.ax.set_xlabel('Probability', labelpad=2)

    # add labels
    ax.set_yticks(range(network.max_labels)[:len(output[index])])
    ax.set_yticklabels(output[index])

    ax.set_xticks(range(network.latent_dim)[:len(sequence2txt(data.test_X, index, data.tokenizer))])
    ax.set_xticklabels(sequence2txt(data.test_X, index, data.tokenizer), rotation=45)

    ax.set_xlabel('Input text')
    ax.set_ylabel('Category')
    ax.grid()

    f.show()
    
visualize_attention(np.argmin([len(x.split(" ")) for x in data.raw_test_X.values]), network, data, probabilities, predictions.predicted_labels)


***Zadanie 6*** Napisz funkcję, która zbierze najważniejsze słowa dla wpisów należących do danej kategorii (przewidzianych). Funkcja powinna zwracać słownik, w której kluczami są słowa, a wartościami jest lista wartości odpowiadających wagom.
- znajdź odpowiednie id ofert z danej kategorii
- dla każdej z tych ofert zbierz najważniejsze dla niej słowa i ich wagi 
- zapisz każdy wynik w słowniku, w którym klucze to słowa, a wartości to lista wag atencji dla poszczególnych słów

In [ ]:
def get_most_significant_word_category(category, data, probabilities, predictions, top_n=10):
    relevant_offers_ids = #znajdź odpowiednie id ofert 
    relevant_probabilities_args_sorted = #posortowane wg ważności indeksy słów dla danych ofert
    words = {}
    
    for i,offer in enumerate(data.test_X[relevant_offers_ids]):
        curr_words = #jakie są słowa które są najważniejsze
        for j,x in enumerate(curr_words):
            #sprawdź czy to słowo jest już w naszym słowniku, jeśli tak, to dodaj tę wagę, jeśli nie, to stwórz pustą listę               
    return words  


def prepare_for_wordcloud_format(*args):
    summed_words_importance_dict = {}
    for dictionary in args:
        for key in dictionary.keys():
            summed_words_importance_dict[key] = summed_words_importance_dict.get(key, 0)
            summed_words_importance_dict[key] += int(np.ceil(10*sum(dictionary[key])))
    wordcloud_format_list = [(x[0] + " ")*x[1] for x in sorted(summed_words_importance_dict.items(), key=lambda x: x[1], reverse=True)]
    wordcloud_format_string = "".join(wordcloud_format_list)
    return wordcloud_format_string

def generate_wordcloud(wordcloud_string):
    wc = WordCloud(collocations=False, width=800, height=500, background_color='#eee8ef', colormap='nipy_spectral').generate(wordcloud_string)
    fig = plt.figure(figsize=(30,20))
    img = plt.imshow(wc, aspect='auto')
    plt.axis('off')
    plt.show()
    return 


generate_wordcloud(prepare_for_wordcloud_format(get_most_significant_word_category("Sprzedaż", data, probabilities, predictions.predictions)))